In [4]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
import pandas as pd
import numpy as np

# own Modules 
from models_sub_net_ls import LstmMse_LatentSpace, LstmMle_LatentSpace, AnalysisLayer
from models_mse import LstmMse
from models_mle import LstmMle_1, LstmMle_2, LstmMle_3
from data_preperator import DataPreperatorPrediction
from data_set import DataSet
from predictor import PredictorMse, PredictorMle, PredictorMseLatentSpaceAnalyser, PredictorMleLatentSpaceAnalyser, PredictorMleSpecial

## Take care of these things before training:
- Select correct path and define droped_features
- Change parameter of model
- Change filed_location

## Parameters phm data

In [6]:
param = {
    "data" : {
        "path" : '../../data/phm_data_challenge/recipe/dataset_for_each_recipe/test/test_recipe_67.csv',
        "droped_feature" : ["stage", "Lot", "runnum", "recipe", "recipe_step",
                            "up time", "ongoing time", 
                            "ETCHSOURCEUSAGE", "ETCHAUXSOURCETIMER", 
                            "ETCHAUX2SOURCETIMER", "FIXTURESHUTTERPOSITION", "ROTATIONSPEED"
                           ],
        "features_not_to_scale": []
    },
    "model" : {
        "path" : "../../models/MLE_model/phm_data_recipe_67Fold_xx_InputSize12_LayerLstm1_HiddenLstm15_HiddenFc75_Seq25.pt",
        "input_size" : 12,
        "n_hidden_lstm" : 15,
        "n_hidden_fc_1" : 75,
        "n_hidden_fc_2" : 25,
        "sequence_size" : 100,
        "batch_size" : 50,
        "lstm_layer" : 1,
        "dropout_rate_lstm": 0.0,
        "dropout_rate_fc": 0.2,
        "K":1
    },
        "anomaly_detection" : {
        "threshold_anomaly" : [2,20], # Individual Threshold for each Sensor
        "no_standard_deviation" : 2
    },
    "results": {
        "path" : "../visualisation/files/prediction/MLE/phm_recipe_67.csv",
    }
}

## Parameters artifical data

In [14]:
param = {
    "data" : {
        "path" : '../../data/artifical_signals/artifical_2_signals_errors.csv',
        "droped_feature" : ["anomaly"
                           ],
        "features_not_to_scale": []
    },
    "model" : {
        "path" : "../../models/MSE_model/artifical_data_Fold_xx_InputSize2_LayerLstm1_HiddenLstm15_HiddenFc75_Seq25.pt",
        "input_size" : 2,
        "n_hidden_lstm" : 15,
        "n_hidden_fc_1" : 75,
        "n_hidden_fc_2" : 25,
        "sequence_size" : 100,
        "batch_size" : 50,
        "lstm_layer" : 1,
        "dropout_rate_lstm": 0.0,
        "dropout_rate_fc": 0.2,
        "K":1
    },
        "anomaly_detection" : {
        "threshold_anomaly" : [0.75,10], # Individual Threshold for each Sensor
        "no_standard_deviation" : 2
    },
    "results": {
        "path" : "../visualisation/files/prediction/MSE/artifical_2_signals.csv",
    }
}

## Parameters cpps data

In [3]:
param = {
    "data" : {
        "path" : '../../data/cpps_degradation/large_degeneration/cpps_data_large_degeneration_test.csv',
        "droped_feature" : ["status"
                           ],
        "features_not_to_scale": []
    },
    "model" : {
        "path" : "../../models/MSE_model/cpps_data_large_degradationFold_xx_InputSize10_LayerLstm1_HiddenLstm15_HiddenFc75_Seq25.pt",
        "input_size" : 10,
        "n_hidden_lstm" : 15,
        "n_hidden_fc_1" : 75,
        "n_hidden_fc_2" : 25,
        "sequence_size" : 100,
        "batch_size" : 50,
        "lstm_layer" : 1,
        "dropout_rate_lstm": 0.0,
        "dropout_rate_fc": 0.2,
        "K":1
    },
        "anomaly_detection" : {
        "threshold_anomaly" : [0.5,0.5,0.6,0.4,0.5,0.6,0.5,0.75,0.5,0.15],
        "no_standard_deviation" : 2
    },
    "results": {
        "path" : "../visualisation/files/prediction/MSE/cpps_large_degradation.csv", 
    }
}

## Standarize Data
First we have to apply normalisation to data. That is because the model works on the representation given by its input vectors. The scale of those numbers is part of the representation.
We should apply the exact same scaling as for training data. That means storing the scale and offset used with your training data, and using that again. <br>
__The mean and variance for each feature of the training data with which the model was trained (stake: 0.75):__

### Mean and variance from phm dataset - recipe 66

In [26]:
mean_training_data =[]
var_training_data =[]

### Mean and variance from phm dataset - recipe 67 (29.219 samples)

In [2]:
mean_training_data = [0.00236702, 0.3953089,0.48772743 ,0.3857511, 0.49987399, 0.06291772, -0.03491417,
                      0.43371134,  0.23365129, -0.06136357, -0.12245359,  0.2516167]
var_training_data =[3.62155978e-04, 7.33502893e-01, 8.30760891e-01, 7.06805763e-01, 9.44752420e-01,
                    4.07861536e-01, 1.61079596e-01, 8.46321709e-01, 3.65172841e-01, 6.37131077e-01,
                    4.60727666e-11, 9.93519995e-01]

### Mean and variance from phm dataset - recipe 68

In [26]:
mean_training_data =[]
var_training_data =[]

### Mean and variance from artifical dataset

In [15]:
mean_training_data= [-0.00526595, -0.00968424]
var_training_data = [49.30277603, 0.4232726 ]

### Mean and Variance form cpps dataset

In [4]:
mean_training_data = [-0.06259308,  0.03496554,  0.07455726,  0.08494991, -0.00575792, -0.00922697, -0.04273393,  0.06988059, -0.03368263,  0.03892771]
var_training_data =[0.14422845, 0.10077938, 0.06166098, 0.06988583, 0.05482907, 0.07040927, 0.04958053, 0.13373154, 0.0509202, 0.02644641]

## Create DataLoader

In [7]:
data_preperator = DataPreperatorPrediction(path=param['data']['path'], 
                                           ignored_features = param["data"]["droped_feature"],
                                           mean_training_data=mean_training_data, 
                                           var_training_data=var_training_data, 
                                           first_order_difference=False 
                                          )                                  
preprocessed_data = data_preperator.prepare_data()
print(preprocessed_data.shape)

dataset = DataSet(preprocessed_data, 
                  timesteps=param["model"]["sequence_size"])
data_loader = DataLoader(dataset, 
                         batch_size=param['model']['batch_size'], 
                         num_workers=0, 
                         shuffle=False, 
                         drop_last=True)

(488363, 13)


In [8]:
for batch_idx, data in enumerate(data_loader):
    x,y = data
    print('Data of batch: {}'.format(batch_idx))
    print("Size of input data: {}".format(x.size()))
    print("Size of target data: {}".format(y.size()))
    if batch_idx >=1: break

Data of batch: 0
Size of input data: torch.Size([50, 100, 13])
Size of target data: torch.Size([50, 13])
Data of batch: 1
Size of input data: torch.Size([50, 100, 13])
Size of target data: torch.Size([50, 13])


## Define Model and load Parameters of trained model
### Model for MSE 

In [7]:
model = LstmMse(batch_size=param['model']['batch_size'], 
                input_dim=param['model']['input_size'], 
                n_hidden_lstm=param['model']['n_hidden_lstm'], 
                n_hidden_fc=param['model']['n_hidden_fc_1'], 
                n_layers=param['model']['lstm_layer'], 
                dropout_rate_lstm= param['model']['dropout_rate_lstm'],
                dropout_rate_fc= param['model']['dropout_rate_fc']
                )

checkpoint = torch.load(param["model"]["path"])
model.load_state_dict(checkpoint['model_state_dict'])

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

### Model for MLE

In [9]:
model = LstmMle_1(batch_size=param['model']['batch_size'], 
                 input_dim=param['model']['input_size'], 
                 n_hidden_lstm=param['model']['n_hidden_lstm'], 
                 n_hidden_fc=param['model']['n_hidden_fc_1'], 
                 n_layers=param['model']['lstm_layer'], 
                 dropout_rate_lstm= param['model']['dropout_rate_lstm'],
                 dropout_rate_fc= param['model']['dropout_rate_fc'],
                 K = param['model']['K'])

checkpoint = torch.load(param["model"]["path"])
model.load_state_dict(checkpoint['model_state_dict'])

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [ ]:
model = LstmMle_3(batch_size=param['model']['batch_size'], 
                 input_dim=param['model']['input_size'], 
                 n_hidden_lstm=param['model']['n_hidden_lstm'], 
                 n_hidden_fc_1=param['model']['n_hidden_fc_1'], 
                 n_hidden_fc_2=param['model']['n_hidden_fc_2'],
                 n_layers=param['model']['lstm_layer'], 
                 dropout_rate_lstm= param['model']['dropout_rate_lstm'],
                 dropout_rate_fc= param['model']['dropout_rate_fc'],
                 K = param['model']['K'])

checkpoint = torch.load(param["model"]["path"])
model.load_state_dict(checkpoint['model_state_dict'])

## Initialize Predictor
### Predictor for MSE Model

In [10]:
predictor = PredictorMse(model=model,
                         path_data=param["data"]["path"],
                         columns_to_ignore=param["data"]["droped_feature"],
                         threshold_anomaly=param["anomaly_detection"]["threshold_anomaly"]
                         )

### Predictor for MLE Model

In [11]:
predictor = PredictorMle(model=model,
                         path_data=param["data"]["path"],
                         columns_to_ignore=param["data"]["droped_feature"],
                         threshold_anomaly=param["anomaly_detection"]["threshold_anomaly"],
                         no_standard_deviation=param["anomaly_detection"]["no_standard_deviation"]
                         )

## Predict

In [12]:
print("Start predicting.")
# Write header
with open(param["results"]["path"], "a+") as file:
            [file.write(column+";") for column in predictor.create_column_names_result()]
            file.write("\n")

for batch_number, (input_data, target_data) in enumerate(data_loader):
    # Predict sensor values in mini-batches
    batch_results = predictor.predict(input_data, target_data)
    
    # Write results to csv file
    with open(param["results"]["path"], "a") as file:
        for batch in batch_results:
            # Each result component of a singe prediction (ID, target, prediction, loss, latent space ...) is stored in lists
            # thus we have to unpack the list and seperate values with ;
            for value in batch:
                file.write(str(value)+";")
            file.write("\n")

    # Print status 
    if (batch_number*param['model']['batch_size'])%5000 == 0:
        print("Current status: " + str(param['model']['batch_size']*batch_number) + " samples are predicted.")

print("End of prediction.")

Start predicting.
Current status: 0 samples are predicted.
Current status: 5000 samples are predicted.
Current status: 10000 samples are predicted.
Current status: 15000 samples are predicted.
Current status: 20000 samples are predicted.
Current status: 25000 samples are predicted.
Current status: 30000 samples are predicted.
Current status: 35000 samples are predicted.
Current status: 40000 samples are predicted.
Current status: 45000 samples are predicted.
Current status: 50000 samples are predicted.
Current status: 55000 samples are predicted.
Current status: 60000 samples are predicted.
Current status: 65000 samples are predicted.
Current status: 70000 samples are predicted.
Current status: 75000 samples are predicted.
Current status: 80000 samples are predicted.
Current status: 85000 samples are predicted.
Current status: 90000 samples are predicted.
Current status: 95000 samples are predicted.
Current status: 100000 samples are predicted.
Current status: 105000 samples are predic

## Tag anomalous samples

In [13]:
results_prediction = pd.read_csv(param["results"]["path"], sep=";")
# Values of column "loss" are exponentially smoothed and stored in a new column "smoothed loss"
# New column "anomaly" is created and sample is taged with 1 if anomalous behaviour (if smoothed loss is over threshold)
results = predictor.detect_anomaly(results_prediction)

In [14]:
results.head()

,ID,IONGAUGEPRESSURE target,ETCHBEAMVOLTAGE target,ETCHBEAMCURRENT target,ETCHSUPPRESSORVOLTAGE target,ETCHSUPPRESSORCURRENT target,FLOWCOOLFLOWRATE target,FLOWCOOLPRESSURE target,ETCHGASCHANNEL1READBACK target,ETCHPBNGASREADBACK target,...,Anomaly Sensor_3,Anomaly Sensor_4,Anomaly Sensor_5,Anomaly Sensor_6,Anomaly Sensor_7,Anomaly Sensor_8,Anomaly Sensor_9,Anomaly Sensor_10,Anomaly Sensor_11,Anomaly Sensor_12
0,101.0,-0.142369,-0.189067,-0.271174,-0.214045,-0.358999,-0.543526,-0.353632,0.327329,0.206600,...,0,0,0,0,0,0,0,0,0,0
1,102.0,-0.142369,-0.189266,-0.270474,-0.214155,-0.356171,-0.549200,-0.353632,0.341207,0.206600,...,0,0,0,0,0,0,0,0,0,0
2,103.0,-0.142369,-0.189266,-0.266971,-0.214100,-0.344493,-0.549200,-0.353632,0.341207,0.206600,...,0,0,0,0,0,0,0,0,0,0
3,104.0,-0.142369,-0.189200,-0.269071,-0.213990,-0.355817,-0.537146,-0.353632,0.328964,0.210369,...,0,0,0,0,0,0,0,0,0,0
4,105.0,-0.142369,-0.188933,-0.265804,-0.213880,-0.345910,-0.537146,-0.353632,0.328964,0.210369,...,0,0,0,0,0,0,0,0,0,0


## Combine prediction data with data which was not consider for inference

In [15]:
original_sensor_data = pd.read_csv(param["data"]["path"])
data_of_droped_feature = original_sensor_data.loc[:, param["data"]["droped_feature"]+["ID"]]
complete_data = results.merge(right=data_of_droped_feature, how="inner", on="ID")

## Save data to csv file

In [16]:
complete_data.to_csv(param["results"]["path"], sep=";", index=False)